In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class GATLayer(nn.Module):
    def __init__(self):
        super(GATLayer, self).__init__()
    def forward(self, input, adj):
        print("")

In [3]:
in_features = 5
out_features = 2
nb_nodes = 3

W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
nn.init.xavier_uniform_(W.data, gain=1.414)

input = torch.rand(nb_nodes, in_features)

# Linear transformation
h = torch.mm(input, W)
N = h.size()[0]

print(h.shape)

torch.Size([3, 2])


In [6]:
a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
nn.init.xavier_uniform_(a.data, gain=1.414)
print(a.shape)
leakyRelu = nn.LeakyReLU(0.2)
a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)],
                   dim=1).view(N, -1, 2 * out_features)

torch.Size([4, 1])


In [15]:
a_input.shape

torch.Size([3, 3, 4])

In [9]:
h

tensor([[0.0562, 1.1620],
        [0.9626, 0.6192],
        [0.3122, 0.1611]], grad_fn=<MmBackward>)

In [10]:
h.repeat(1, N)

tensor([[0.0562, 1.1620, 0.0562, 1.1620, 0.0562, 1.1620],
        [0.9626, 0.6192, 0.9626, 0.6192, 0.9626, 0.6192],
        [0.3122, 0.1611, 0.3122, 0.1611, 0.3122, 0.1611]],
       grad_fn=<RepeatBackward>)

In [11]:
h.repeat(1, N).view(N*N, -1)

tensor([[0.0562, 1.1620],
        [0.0562, 1.1620],
        [0.0562, 1.1620],
        [0.9626, 0.6192],
        [0.9626, 0.6192],
        [0.9626, 0.6192],
        [0.3122, 0.1611],
        [0.3122, 0.1611],
        [0.3122, 0.1611]], grad_fn=<ViewBackward>)